In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)
!nvcc --version
# !pip install cloud-tpu-client==0.10 -q
# !pip install https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl -q

1.9.0+cu102
10.2
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [2]:
%load_ext autoreload
%autoreload 2 

In [3]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet

/content/gdrive/MyDrive/Colab Notebooks/LidarObjectDetection/VoxelNet


In [5]:
%%writefile train.py
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils import data

import time
from model import VoxelNet
from loss import VoxelNetLoss
from config import config as cfg
from data.dataset import KittiDataset
import pathlib, os
import numpy as np

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight.data)
        m.bias.data.zero_()

def detection_collate(batch):
  voxel_features = []
  voxel_coords = []
  pos_equal_one = []
  neg_equal_one = []
  targets = []
  images = []
  calibs = []
  ids = []

  for i, sample in enumerate(batch):
    voxel_features.append(sample['voxel_features'])
    voxel_coords.append(
        np.pad(sample['voxel_coords'], ((0,0), (1,0)), mode='constant',
               constant_values=i
        )
    )
    pos_equal_one.append(sample['pos_equal_one'])
    neg_equal_one.append(sample['neg_equal_one'])
    targets.append(sample['target'])
    images.append(sample['rgb'])
    calibs.append(sample['calib'])
    ids.append(sample['file_id'])

    return np.concatenate(voxel_features), np.concatenate(voxel_coords), \
           np.array(pos_equal_one), np.array(neg_equal_one), \
           np.array(targets), images, calibs, ids

# model
net = VoxelNet()
device = torch.device("cuda")

print("\n## Hardware type: {}\n".format(device.type))
net.to(device)

# training mode
net.train()

# initialization
print('Initializing weights...')
net.apply(weights_init)

# define optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# loss function
loss = VoxelNetLoss(alpha=1.5, beta=1)



# dataset
print("## data path:", pathlib.Path(os.getcwd()).absolute()/"data/training")
dataset=KittiDataset(pathlib.Path(os.getcwd()).absolute()/"data/training", cfg)
data_loader = data.DataLoader(dataset, batch_size=cfg.N, shuffle=True, 
                              num_workers=2, collate_fn=detection_collate,
                              pin_memory=False)
# training process
batch_iterator = None
epoch_size = len(dataset) // cfg.N
print('## epoch size', epoch_size)

for iteration in range(10000):
  if (not batch_iterator) or (iteration % epoch_size == 0):
    batch_iterator = iter(data_loader)

  voxel_features, voxel_coords, pos_equal_one, neg_equal_one, targets, images, calibs, ids= next(batch_iterator)
  
  voxel_features = Variable(torch.cuda.FloatTensor(voxel_features))
  pos_equal_one = Variable(torch.cuda.FloatTensor(pos_equal_one))
  neg_equal_one = Variable(torch.cuda.FloatTensor(neg_equal_one))
  targets = Variable(torch.cuda.FloatTensor(targets))

  optimizer.zero_grad()

  t0 = time.time()
  psm, rm = net(voxel_features, voxel_coords)

  # calculate loss
  conf_loss, reg_loss = loss(rm, psm, pos_equal_one, neg_equal_one, targets)
  loss_sum = conf_loss + reg_loss
  # backward
  loss_sum.backward()
  optimizer.step()

  t1 = time.time()
  print('Timer: %.4f sec.' % (t1 - t0))
  # print(loss_sum)
  # print(conf_loss)
  # print(reg_loss)
  print('iter ' + repr(iteration) + ' || Loss: %.4f || Conf Loss: %.4f || Loc Loss: %.4f' % \
        (loss_sum.item(), conf_loss.item(), reg_loss.item()))


Overwriting train.py


In [6]:
!sudo rm -rf ~/.nv
!python train.py


## Hardware type: cuda

Initializing weights...
## data path: /content/gdrive/My Drive/Colab Notebooks/LidarObjectDetection/VoxelNet/data/training
## epoch size 1884
tcmalloc: large alloc 1441792000 bytes == 0x5558681aa000 @  0x7f2f503fdb6b 0x7f2f5041d379 0x7f2ef4c2b26e 0x7f2ef4c2c9e2 0x7f2f38496b49 0x7f2f38497897 0x7f2f38873d89 0x7f2f38fd8b9a 0x7f2f38fbbcbe 0x7f2f38bc0a05 0x7f2f38877df0 0x7f2f3914ab94 0x7f2f38fa8bf4 0x7f2f38fbefb7 0x7f2f38c1a171 0x7f2f4b3eb915 0x5557f6eead54 0x5557f6eeaa50 0x5557f6f5f105 0x5557f6eec30a 0x5557f6f5e7f0 0x5557f6eec849 0x5557f6eecea1 0x5557f6f5bbb5 0x5557f6f594ae 0x5557f6eeca81 0x5557f6f2ffd9 0x5557f6eeb462 0x5557f6f5e715 0x5557f6f594ae 0x5557f6f591b3
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:921: UserWarning: Using a target size (torch.Size([1, 200, 176, 2, 7])) that is different to the input size (torch.Size([2, 200, 176, 2, 7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size

In [ ]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet
!python -m setup build_ext --inplace

/content/gdrive/MyDrive/Colab Notebooks/LidarObjectDetection/VoxelNet
running build_ext
building 'box_overlaps' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/third_party
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-OGiuun/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-OGiuun/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I/usr/include/python3.7m -c ./third_party/box_overlaps.c -o build/temp.linux-x86_64-3.7/./third_party/box_overlaps.o
In file included from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
      